In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
### Import dependencies
# Standard python libraries
import os
# Third party libraries
import pandas as pd
import numpy as np
# Custom packages
from ffpackage.scraping import mfl
from ffpackage.scraping import ourlads
from ffpackage.analysis import analysis
from ffpackage.viz import viz
from appmanager.database import db

In [3]:
# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)


In [4]:
# Retrieve basic info on all players
allPlayers = mfl.get_players()
# Retrieve Fantasy Sharks rankings
sharkRanks = mfl.get_sharkRanks()
# Retrieve Average Draft Positions
adpValues = mfl.get_adp()

# Find player ages
# Retrieve any player dobs who are already in the db
player_dobs = db.get_df('player_dobs')
# Rename the dob columns until I can get to that in the app setup
player_dobs = player_dobs.rename(columns={'PlayerID':'id_mfl', 'DOB':'dob', 'Age':'age'})
# Check for any players whose ages are not already in the db
ageNeeded = allPlayers.loc[~allPlayers['id_mfl'].isin(player_dobs['id_mfl'])]
# If there are players whose ages are not in the db, retrieve those ages from My Fantasy League
if len(ageNeeded)>0:
    # Break player list into chunks small enough for the API server
    # Specify chunk size
    n = 50
    # Slice the df where ages are needed into chunks
    list_df = [ageNeeded['id_mfl'][i:i+n] for i in range(0,ageNeeded['id_mfl'].shape[0],n)]
    # Loop over the chunks
    for i in range(len(list_df)):
        # Join the playerIDs into a string to use in building the API call
        idList = ",".join(list_df[i])
        # Retrieve the dates of birth for these players from My Fantasy League
        new_dobs = mfl.get_playerProfiles(idList)
        # Append these players to the list that already existed in the app's database
        player_dobs = player_dobs.append(new_dobs)
# Convert date columns to datetime
player_dobs['dob'] = pd.to_datetime(player_dobs['dob'])
# Convert Date of Birth to Age using datetime's relativedelta module
player_dobs['age'] = player_dobs['dob'].apply(analysis.calculate_age)
# Drop the dob column
player_dobs = player_dobs.drop(columns='dob')

# Merge all dfs from MyFantasyLeague API
allPlayers = allPlayers.merge(
    player_dobs, on='id_mfl', how='left'
).merge(
    sharkRanks, on='id_mfl', how='left'
).merge(
    adpValues, on='id_mfl', how='left'
)
# Clean sharkRank and ADP columns
# If a player is ranked low enough, they do not get a ranking in FantasySharks or ADP. Replace these with a very low rank
allPlayers['sharkRank'].fillna(3000, inplace=True)
allPlayers['adp'].fillna(3000, inplace=True)

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/scraping/mfl.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'playerName'] = df.loc[:, 'playerName'].str.replace(".", "")
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_8484/4278180581.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get th

In [5]:
# Retrieve position ranks from OurLads
posRanks = ourlads.scrape_depthcharts()

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/scraping/ourlads.py:78: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'playerName'] = df.loc[:, 'playerName'].str.replace(".", "")


In [6]:
### Merge MyFantasyLeague data with OurLads data
allPlayers = allPlayers.merge(posRanks, how='left', on=['playerName', 'team'])
## Clean merged df
# Give defenses a position rank since OurLads does not include them
allPlayers.loc[allPlayers['pos']=='DF', 'posRank'] = "DF1"
# Give any players who were not on the OurLads website a rank of 3
allPlayers.loc[allPlayers['posRank'].isna(), 'posRank'] = allPlayers.loc[allPlayers['posRank'].isna(), 'pos'] + "3"
# Give any injured reserve players a rank of 3
allPlayers.loc[allPlayers['posRank']=="RES", 'posRank'] = allPlayers.loc[allPlayers['posRank']=="RES", 'pos'] + "3"
# Specify all players are in current season
allPlayers['season'] = 2022

In [7]:
### Fetch historical data from app database
prior1 = db.get_df('prior1')
prior2 = db.get_df('prior2')
# Rename columns
prior1 = prior1.rename(columns={'player':'playerName'})
prior2 = prior2.rename(columns={'player':'playerName'})

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [8]:
# Create current_df
# This will mean scraping the ff db site weekly
curr = pd.DataFrame(allPlayers['playerName'])
colList = ['gamesPlayed',
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY','rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM','FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk','defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst','defYdsAgainst'
]
cols = pd.DataFrame(columns=colList)
curr = curr.merge(cols, how='left', left_index=True, right_index=True)
curr.fillna(0, inplace=True)

# Rename all columns in curr
colList = [(x + "_curr") for x in list(curr.columns)]
curr.columns = colList
curr = curr.rename(columns={
       'playerName_curr':'playerName',
       })
curr

,playerName,gamesPlayed_curr,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,defSack_curr,defI_curr,defSaf_curr,defFum_curr,defBlk_curr,defT_curr,defPtsAgainst_curr,defPassYAgainst_curr,defRushYAgainst_curr,defYdsAgainst_curr
0,BUFFALO BILLS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,INDIANAPOLIS COLTS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MIAMI DOLPHINS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NEW ENGLAND PATRIOTS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NEW YORK JETS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,RASHID SHAHEED,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1163,DRE MILLER,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1164,NICK SCIBA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1165,STANLEY BERRYHILL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Merge playerdf, currentdf, prior1, and prior2
allPlayers = allPlayers.merge(curr, how='left', on='playerName').merge(prior1, how='left', on='playerName').merge(prior2, how='left', on='playerName')
# Fill data for players who do not have prior data
allPlayers.fillna(0, inplace=True)

In [10]:
allPlayers.drop_duplicates(subset=['playerName', 'pos'], inplace=True)


In [11]:
# Get schedule
schedule = db.get_df('schedule')
# Merge in opponents
allPlayers = allPlayers.merge(schedule, how='left', on='team')
allPlayers

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


,id_mfl,playerName,pos,team,age,sharkRank,adp,posRank,PR,KR,...,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2,week,opponent
0,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.0,0.6875,0.03125,0.15625,20.4375,202.875,116.625,319.5,1.0,LAR
1,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.0,0.6875,0.03125,0.15625,20.4375,202.875,116.625,319.5,3.0,MIA
2,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.0,0.6875,0.03125,0.15625,20.4375,202.875,116.625,319.5,4.0,BAL
3,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.0,0.6875,0.03125,0.15625,20.4375,202.875,116.625,319.5,6.0,KCC
4,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.0,0.6875,0.03125,0.15625,20.4375,202.875,116.625,319.5,9.0,NYJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12279,16097,KYRIC MCGOWAN,WR,WAS,23.0,3000.0,3000.0,WR3,0,0,...,0.0,0.0000,0.00000,0.00000,0.0000,0.000,0.000,0.0,9.0,MIN
12280,16097,KYRIC MCGOWAN,WR,WAS,23.0,3000.0,3000.0,WR3,0,0,...,0.0,0.0000,0.00000,0.00000,0.0000,0.000,0.000,0.0,12.0,ATL
12281,16097,KYRIC MCGOWAN,WR,WAS,23.0,3000.0,3000.0,WR3,0,0,...,0.0,0.0000,0.00000,0.00000,0.0000,0.000,0.000,0.0,15.0,NYG
12282,16097,KYRIC MCGOWAN,WR,WAS,23.0,3000.0,3000.0,WR3,0,0,...,0.0,0.0000,0.00000,0.00000,0.0000,0.000,0.000,0.0,17.0,CLE


In [12]:
# Get opponent historical data
# select only defenses
allDef = allPlayers.loc[allPlayers['pos']=='DF']
allDef

,id_mfl,playerName,pos,team,age,sharkRank,adp,posRank,PR,KR,...,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2,week,opponent
0,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.00000,0.6875,0.03125,0.15625,20.4375,202.875,116.6250,319.5000,1.0,LAR
1,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.00000,0.6875,0.03125,0.15625,20.4375,202.875,116.6250,319.5000,3.0,MIA
2,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.00000,0.6875,0.03125,0.15625,20.4375,202.875,116.6250,319.5000,4.0,BAL
3,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.00000,0.6875,0.03125,0.15625,20.4375,202.875,116.6250,319.5000,6.0,KCC
4,0501,BUFFALO BILLS,DF,BUF,0.0,332.0,3000.0,DF1,0,0,...,0.00000,0.6875,0.03125,0.15625,20.4375,202.875,116.6250,319.5000,9.0,NYJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0532,HOUSTON TEXANS,DF,HOU,0.0,696.0,3000.0,DF1,0,0,...,0.03125,0.4375,0.06250,0.09375,27.7500,248.125,151.8125,399.9375,9.0,PHI
540,0532,HOUSTON TEXANS,DF,HOU,0.0,696.0,3000.0,DF1,0,0,...,0.03125,0.4375,0.06250,0.09375,27.7500,248.125,151.8125,399.9375,11.0,WAS
541,0532,HOUSTON TEXANS,DF,HOU,0.0,696.0,3000.0,DF1,0,0,...,0.03125,0.4375,0.06250,0.09375,27.7500,248.125,151.8125,399.9375,13.0,CLE
542,0532,HOUSTON TEXANS,DF,HOU,0.0,696.0,3000.0,DF1,0,0,...,0.03125,0.4375,0.06250,0.09375,27.7500,248.125,151.8125,399.9375,15.0,KCC


In [13]:
# Get current defensive scores
currDef = allDef.copy()
# Select only relevant columns
currDef = currDef[['team', 'week',
       'defSack_curr', 'defI_curr',
       'defSaf_curr', 'defFum_curr', 'defBlk_curr',
       'defT_curr', 'defPtsAgainst_curr', 'defPassYAgainst_curr',
       'defRushYAgainst_curr', 'defYdsAgainst_curr']]

# Get prior defensive scores
priorDef = allDef.copy()
# Select only relevant columns
priorDef = priorDef[['team', 'week',
       'defSack_prior1', 'defI_prior1',
       'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1',
       'defT_prior1', 'defPtsAgainst_prior1', 'defPassYAgainst_prior1',
       'defRushYAgainst_prior1', 'defYdsAgainst_prior1']]
# Merge the two defensive dfs
allDef = currDef.merge(priorDef, how='left', on=['team', 'week'])

# Rename all columns in allDef
colList = [(x + "_opp") for x in list(allDef.columns)]
allDef.columns = colList
allDef = allDef.rename(columns={
       'team_opp':'opponent',
       'week_opp':'week'
       })

In [14]:
# Connect opponents to defenses
allPlayers = allPlayers.merge(allDef, how='left', on=['opponent', 'week'])
print(len(allPlayers))

12284


In [15]:

# Ensure df has correct columns in the correct order
allPlayers = allPlayers[[
    'id_mfl',
    'season',
    'week',
    'team',
    'playerName',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent',
    'passA_curr',
    'passC_curr',
    'passY_curr',
    'passT_curr',
    'passI_curr',
    'pass2_curr',
    'rushA_curr',
    'rushY_curr',
    'rushT_curr',
    'rush2_curr',
    'recC_curr',
    'recY_curr',
    'recT_curr',
    'rec2_curr',
    'fum_curr',
    'XPA_curr',
    'XPM_curr',
    'FGA_curr',
    'FGM_curr',
    'FG50_curr',
    'defSack_curr',
    'defI_curr',
    'defSaf_curr',
    'defFum_curr',
    'defBlk_curr',
    'defT_curr',
    'defPtsAgainst_curr',
    'defPassYAgainst_curr',
    'defRushYAgainst_curr',
    'defYdsAgainst_curr',
    'gamesPlayed_curr',
    'gamesPlayed_prior1',
    'passA_prior1',
    'passC_prior1',
    'passY_prior1',
    'passT_prior1',
    'passI_prior1',
    'pass2_prior1',
    'rushA_prior1',
    'rushY_prior1',
    'rushT_prior1',
    'rush2_prior1',
    'recC_prior1',
    'recY_prior1',
    'recT_prior1',
    'rec2_prior1',
    'fum_prior1',
    'XPA_prior1',
    'XPM_prior1',
    'FGA_prior1',
    'FGM_prior1',
    'FG50_prior1',
    'defSack_prior1',
    'defI_prior1',
    'defSaf_prior1',
    'defFum_prior1',
    'defBlk_prior1',
    'defT_prior1',
    'defPtsAgainst_prior1',
    'defPassYAgainst_prior1',
    'defRushYAgainst_prior1',
    'defYdsAgainst_prior1',
    'gamesPlayed_prior2',
    'passA_prior2',
    'passC_prior2',
    'passY_prior2',
    'passT_prior2',
    'passI_prior2',
    'pass2_prior2',
    'rushA_prior2',
    'rushY_prior2',
    'rushT_prior2',
    'rush2_prior2',
    'recC_prior2',
    'recY_prior2',
    'recT_prior2',
    'rec2_prior2',
    'fum_prior2',
    'XPA_prior2',
    'XPM_prior2',
    'FGA_prior2',
    'FGM_prior2',
    'FG50_prior2',
    'defSack_prior2',
    'defI_prior2',
    'defSaf_prior2',
    'defFum_prior2',
    'defBlk_prior2',
    'defT_prior2',
    'defPtsAgainst_prior2',
    'defPassYAgainst_prior2',
    'defRushYAgainst_prior2',
    'defYdsAgainst_prior2',
    'defSack_curr_opp',
    'defI_curr_opp',
    'defSaf_curr_opp',
    'defFum_curr_opp',
    'defBlk_curr_opp',
    'defT_curr_opp',
    'defPtsAgainst_curr_opp',
    'defPassYAgainst_curr_opp',
    'defRushYAgainst_curr_opp',
    'defYdsAgainst_curr_opp',
    'defSack_prior1_opp',
    'defI_prior1_opp',
    'defSaf_prior1_opp',
    'defFum_prior1_opp',
    'defBlk_prior1_opp',
    'defT_prior1_opp',
    'defPtsAgainst_prior1_opp',
    'defPassYAgainst_prior1_opp',
    'defRushYAgainst_prior1_opp',
    'defYdsAgainst_prior1_opp']]

In [ ]:
positionList = ['QB', 'RB', 'WR', 'TE', 'PK', 'DF']
predStats = pd.DataFrame()
for pos in positionList:
    predXpos = predictions.makePredictions(allPlayers, pos=pos)
    predStats = pd.concat([predStats, predXpos])

In [16]:
labels = [
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY', 'rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM', 'FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk', 'defT', 
    'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst', 'defYdsAgainst'  
]

features = [
    'week', 'age', 
    'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 
    'rushA_curr', 'rushY_curr', 'rushT_curr', 'rush2_curr', 
    'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 
    'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 'FG50_curr', 
    'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 
    'defPtsAgainst_curr', 'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 
    'gamesPlayed_curr', 
    'gamesPlayed_prior1', 
    'passA_prior1', 'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 
    'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 'rush2_prior1', 
    'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 
    'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 'FGM_prior1', 'FG50_prior1', 
    'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 
    'defPtsAgainst_prior1', 'defPassYAgainst_prior1', 'defRushYAgainst_prior1', 'defYdsAgainst_prior1', 
    'gamesPlayed_prior2', 
    'passA_prior2', 'passC_prior2', 'passY_prior2', 'passT_prior2', 'passI_prior2', 'pass2_prior2', 
    'rushA_prior2', 'rushY_prior2', 'rushT_prior2', 'rush2_prior2', 
    'recC_prior2', 'recY_prior2', 'recT_prior2', 'rec2_prior2', 'fum_prior2', 
    'XPA_prior2', 'XPM_prior2', 'FGA_prior2', 'FGM_prior2', 'FG50_prior2', 
    'defSack_prior2', 'defI_prior2', 'defSaf_prior2', 'defFum_prior2', 'defBlk_prior2', 'defT_prior2', 
    'defPtsAgainst_prior2', 'defPassYAgainst_prior2', 'defRushYAgainst_prior2', 'defYdsAgainst_prior2', 
    'defSack_curr_opp', 'defI_curr_opp', 'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp', 'defT_curr_opp', 
    'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp', 'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp', 
    'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp', 'defFum_prior1_opp', 'defBlk_prior1_opp', 'defT_prior1_opp', 
    'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp', 'defRushYAgainst_prior1_opp', 'defYdsAgainst_prior1_opp', 
    'pos', 'posRank'
]

In [17]:
### WR Predictions
# Read player model and ages
xl2 = allPlayers.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['WR1', 'WR2', 'WR3'])]
xl2 = xl2.loc[xl2.pos=='WR']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'playerName',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

# Check if there were the correct number of posRanks in the dataset
for rank in ["posRank_WR1", "posRank_WR2", "posRank_WR3"]:
    if rank not in list(X.columns):
        X[rank] = 0

# Make sure we have the necessary columns
X = X[['week', 'age', 'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 'rushA_curr', 'rushY_curr', 
'rushT_curr', 'rush2_curr', 'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 
'FG50_curr', 'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 'defPtsAgainst_curr', 
'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 'gamesPlayed_curr', 'gamesPlayed_prior1', 'passA_prior1', 
'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 
'rush2_prior1', 'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 
'FGM_prior1', 'FG50_prior1', 'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 
'defPtsAgainst_prior1', 'defPassYAgainst_prior1', 'defRushYAgainst_prior1', 'defYdsAgainst_prior1', 'gamesPlayed_prior2', 'passA_prior2', 
'passC_prior2', 'passY_prior2', 'passT_prior2', 'passI_prior2', 'pass2_prior2', 'rushA_prior2', 'rushY_prior2', 'rushT_prior2', 
'rush2_prior2', 'recC_prior2', 'recY_prior2', 'recT_prior2', 'rec2_prior2', 'fum_prior2', 'XPA_prior2', 'XPM_prior2', 'FGA_prior2', 
'FGM_prior2', 'FG50_prior2', 'defSack_prior2', 'defI_prior2', 'defSaf_prior2', 'defFum_prior2', 'defBlk_prior2', 'defT_prior2', 
'defPtsAgainst_prior2', 'defPassYAgainst_prior2', 'defRushYAgainst_prior2', 'defYdsAgainst_prior2', 'defSack_curr_opp', 'defI_curr_opp', 
'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp', 'defT_curr_opp', 'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp', 
'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp', 'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp', 'defFum_prior1_opp', 
'defBlk_prior1_opp', 'defT_prior1_opp', 'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp', 'defRushYAgainst_prior1_opp', 
'defYdsAgainst_prior1_opp', 'pos_WR', 'posRank_WR1', 'posRank_WR2', 'posRank_WR3']]

#load saved model
regressor = load('../models/rfmodel_WR1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','playerName','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
WRdf = header.merge(c, left_index=True, right_index=True)

/Users/travisharry/opt/anaconda3/envs/ff_demo2/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/travisharry/opt/anaconda3/envs/ff_demo2/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
